In [1]:
from numpy.random import seed
seed(1)
import tensorflow 
tensorflow.random.set_seed(1) 
import pandas as pd
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from collections import Counter
from tensorflow import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv1D, MaxPooling1D, Convolution1D
from keras.optimizers import Adadelta
from sklearn import metrics
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import json
import time
import datetime
import numpy as np

2025-05-27 07:48:32.754093: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-05-27 07:48:32.754106: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[nltk_data] Downloading package stopwords to /home/alice/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def remove_stopwords(text):
    all_words = text.split(" ")
    clean_text = [i for i in all_words if i not in stopwords and i!=""]
    return " ".join(clean_text)

def remove_html_tags(text):
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

def lower_texts(text):
    return text.lower()

def clean_text(text):
    clean_text = remove_stopwords(text)
    clean_text = remove_html_tags(clean_text)
    clean_text = lower_texts(clean_text)
    return clean_text

In [3]:
from keras.callbacks import EarlyStopping 
from tensorflow.keras.initializers import Constant
from tensorflow.keras.initializers import GlorotUniform

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Activation, Flatten, Dropout, Dense
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.optimizers import Adadelta

def create_model_age(filters=[100], kernel_size=[50], strides=[100], 
                     dropout_rate=0.5, pool_size=[5], dense_units=100, max_len=1000):

    model = Sequential()

    # First convolutional layer with deterministic initialization
    model.add(Conv1D(filters=filters[0], 
                     kernel_size=kernel_size[0],
                     strides=strides[0], 
                     activation='relu', 
                     input_shape=(max_len, 1),
                     kernel_initializer=GlorotUniform(seed=42)))

    # MaxPooling and activation layers
    for i in range(len(pool_size)):
        model.add(MaxPooling1D(pool_size=pool_size[i], strides=1))
        model.add(Activation('relu'))

    model.add(Flatten())

    # Dropout layer without a seed (as it's deprecated for Dropout in TensorFlow 2.x)
    if dropout_rate is not None:
        model.add(Dropout(dropout_rate))
    
    # Dense layers with fixed initializers
    model.add(Dense(units=dense_units, activation='relu', kernel_initializer=GlorotUniform(seed=42)))
    model.add(Dense(units=3, activation='softmax', kernel_initializer=GlorotUniform(seed=42)))

    # Compile the model
    model.compile(loss='categorical_crossentropy', 
                  optimizer=Adadelta(learning_rate=1), 
                  metrics=['accuracy'])

    return model


In [5]:
def formatTime(seg):
    return str(datetime.timedelta(seconds=seg))

In [6]:
def oversampling(X, y):
    X_resampled, y_resampled = SMOTE().fit_resample(X, y)
    return X_resampled, y_resampled
    # return X, y

#X_test_resampled, y_test_resampled = oversampling(X_test, y_test)

In [7]:
df = pd.read_csv("../data/b5-ready.csv")

df["Age"] = df["Age_Group"] - 1

df['Clean_Text'] = df[ 'Text'].apply(lambda x: clean_text(x))

X = df[ 'Clean_Text']
y = df["Age"]

In [24]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=958, stratify=y)
#df_train, df_test = train_test_split(df, test_size=0.2, random_state=958, stratify=y)
df_train = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')

X_train = df_train['Clean_text']
y_train = df_train['Age'].to_numpy()

X_test = df_test['Clean_text']
y_test = df_test['Age'].to_numpy()

num_words = []
for text in X_train.tolist():
    num_words.append(len(text.split()))
mean = sum(num_words)//len(num_words)

train_texts = X_train.tolist()
test_texts = X_test.tolist()
tfidfvec = TfidfVectorizer(max_features = mean, max_df=0.9)
tfidfvec.fit(train_texts)
tfidf_train = tfidfvec.transform(train_texts).toarray()
tfidf_test = tfidfvec.transform(test_texts).toarray()

X_train_resampled, y_train_resampled = oversampling(tfidf_train, y_train)

X_train = X_train_resampled.reshape(X_train_resampled.shape[0],X_train_resampled.shape[1],1)
X_test = tfidf_test.reshape(tfidf_test.shape[0],tfidf_test.shape[1],1)
y_train= keras.utils.to_categorical(y_train_resampled,num_classes=3)
y_test = keras.utils.to_categorical(y_test,num_classes=3)

size = X_test.shape[1]

f1 = []

model = create_model_age(filters=[60], kernel_size=[1], strides=[1], dropout_rate=0.1, pool_size=[4], dense_units = 512, max_len = X_test.shape[1])
callback = [
            EarlyStopping(patience=5, monitor='val_accuracy',  mode='max', restore_best_weights=True),
        ]

model.fit(X_train,y_train,validation_data=(X_test,y_test), batch_size=32, epochs=10, callbacks=callback)

y_pred = model.predict(
        X_test
)

import numpy as np

y_pred_list = [np.argmax(x, axis=-1) for x in y_pred]
y_test_list =[np.argmax(x, axis=-1) for x in y_test]

print(metrics.f1_score(y_test_list, y_pred_list, average='macro'))

Epoch 1/10
15/15 [==============================] - 2s 145ms/step - loss: 1.0875 - accuracy: 0.3841 - val_loss: 1.1013 - val_accuracy: 0.3558
Epoch 2/10
15/15 [==============================] - 2s 140ms/step - loss: 1.0404 - accuracy: 0.4879 - val_loss: 1.0219 - val_accuracy: 0.4519
Epoch 3/10
15/15 [==============================] - 2s 140ms/step - loss: 0.9486 - accuracy: 0.5806 - val_loss: 1.1489 - val_accuracy: 0.3558
Epoch 4/10
15/15 [==============================] - 2s 140ms/step - loss: 0.8836 - accuracy: 0.5828 - val_loss: 0.9663 - val_accuracy: 0.6154
Epoch 5/10
15/15 [==============================] - 2s 138ms/step - loss: 0.7976 - accuracy: 0.6534 - val_loss: 0.9759 - val_accuracy: 0.5096
Epoch 6/10
15/15 [==============================] - 2s 138ms/step - loss: 0.6954 - accuracy: 0.6600 - val_loss: 1.0709 - val_accuracy: 0.4519
Epoch 7/10
15/15 [==============================] - 2s 140ms/step - loss: 0.6118 - accuracy: 0.7616 - val_loss: 1.1786 - val_accuracy: 0.4135
Epoch 

In [42]:
model.save('final-model.keras')

In [25]:
dict_class = pd.read_csv('../data/dict-class.csv', encoding='latin-1')
no_class = pd.read_csv('../data/no-class.csv', encoding='latin-1')

In [26]:
len(y_pred_list), df_test.shape

(104, (104, 9))

In [27]:
df_test = df_test.reset_index(drop=True)
dict_class = dict_class.reset_index(drop=True)

df_test['User_ID'] = df_test['User_ID'].astype(int)
dict_class['User_ID'] = dict_class['User_ID'].astype(int)
df_test['Predict'] = y_pred_list

compare_test = df_test[df_test['User_ID'].isin(dict_class['User_ID'])]

In [28]:
compare_test.shape

(11, 10)

In [29]:
dict_class.columns

Index(['Unnamed: 0.1', 'index', 'Unnamed: 0', 'User_ID', 'age', 'Text',
       'Age_Group', 'Text_TK', 'Age', 'Clean_text', 'glex1', 'glex2', 'glex3',
       'c', 'c50', 'c5025', 'cmean', 'c75', 'c7525'],
      dtype='object')

In [30]:
compare_test = compare_test[['User_ID', 'Age_Group', 'Age', 'Clean_text', f'Predict']]
compare_dict = dict_class[['User_ID', 'c7525']]

compare = compare_test.merge(compare_dict, left_on='User_ID', right_on='User_ID')

In [31]:
metrics.accuracy_score(compare['Age'], compare[f'Predict']), metrics.accuracy_score(compare['Age_Group'], compare['c7525'])

(0.7272727272727273, 0.8181818181818182)

In [32]:
metrics.f1_score(compare['Age'], compare[f'Predict'], average='macro'), metrics.f1_score(compare['Age_Group'], compare['c7525'], average='macro')

(0.7802197802197802, 0.861111111111111)

In [33]:
compare.head()

,User_ID,Age_Group,Age,Clean_text,Predict,c7525
0,276,2,1,b melo sabe programar manja ruby on rails venh...,1,2
1,598,3,2,companheiras banca ciência prezados amigos ami...,2,3
2,1122,1,0,morri dia mãe linda hoje sempre dia agradecer ...,0,1
3,972,1,0,olha machado each resiste usp greve aí coração...,0,1
4,1119,2,1,alive horas volta rumo qualquer esforço caber ...,0,1


In [34]:
compare.shape

(11, 6)

In [35]:
compare['c7525'] = compare['c7525']-1

In [36]:
compare.head()

,User_ID,Age_Group,Age,Clean_text,Predict,c7525
0,276,2,1,b melo sabe programar manja ruby on rails venh...,1,1
1,598,3,2,companheiras banca ciência prezados amigos ami...,2,2
2,1122,1,0,morri dia mãe linda hoje sempre dia agradecer ...,0,0
3,972,1,0,olha machado each resiste usp greve aí coração...,0,0
4,1119,2,1,alive horas volta rumo qualquer esforço caber ...,0,0


In [37]:
merged_df = df_test.merge(compare[['User_ID', 'c7525']], on='User_ID', how='left')

In [38]:
merged_df.shape

(104, 11)

In [39]:
merged_df.head()

,Unnamed: 0.1,Unnamed: 0,User_ID,age,Text,Age_Group,Text_TK,Age,Clean_text,Predict,c7525
0,209,508,584,44.0,"Onde é a fila da encomenda ? # "" Segunda aula...",3,"['Onde', 'é', 'a', 'fila', 'da', 'encomenda', ...",2,onde fila encomenda segunda aula gastronomia m...,2,NaN
1,295,664,758,19.0,"Fofa do meu # Porque eu sou a química # "" Pe...",1,"['Fofa', 'do', 'meu', '#', 'Porque', 'eu', 'so...",0,fofa porque química pensei felicidade começass...,0,NaN
2,108,295,345,23.0,""" "" "" Tem que problematizar sim , tem que disc...",2,"['``', '``', '``', 'Tem', 'que', 'problematiza...",1,problematizar sim discutir sim janeiro saudade...,1,NaN
3,87,232,276,24.0,"$NAME$ $NAME$ B . Melo # "" Sabe programar ? M...",2,"['$', 'NAME', '$', '$', 'NAME', '$', 'B', '.',...",1,b melo sabe programar manja ruby on rails venh...,1,1.0
4,219,521,598,53.0,"Minhas companheiras da Banca da ciência # "" P...",3,"['Minhas', 'companheiras', 'da', 'Banca', 'da'...",2,companheiras banca ciência prezados amigos ami...,2,2.0


In [40]:
merged_df['Predict'] = merged_df['c7525'].combine_first(merged_df['Predict'])

In [41]:
metrics.f1_score(merged_df['Age'], merged_df[f'Predict'], average='macro'), metrics.f1_score(df_test['Age'], df_test['Predict'], average='macro')

(0.6190671651216247, 0.6082214924956015)

In [43]:
metrics.accuracy_score(merged_df['Age'], merged_df[f'Predict']), metrics.accuracy_score(df_test['Age'], df_test['Predict'])

(0.625, 0.6153846153846154)

In [45]:
compare.shape[0]/df_test.shape[0]

0.10576923076923077